# 5. Fitting Models 

 #### 지지지난 시간, pyMC 를 실행하기에 필요한 모델을 만드는 데 필요한 데이터들을 어떻게 구현하는지 알아보았습니다. 
 #### (Stochastic, Deterministic, Potential)

 #### 이번 장에서는, 만들어진 데이터 셋으로 계산 모델을 만들고, pyMC 에서 계산이 가능한 Fitting 종류를 알아보겠습니다.  
 #### (MCMC, MAP, NormApprox, Metropolis, Gibbs) 






## 5.1 Creating Models 
  
 #### 지지난 시간, 동전 던지기에 대한 간단한 실습이 있었는데, MCMC를 돌리기 위한 과정은 
 
 ### 데이터 입력 -> 입력값들 모델링 -> MCMC에 입력 


In [1]:
import pymc as pm
import numpy as np


# data from experiment 
f1=open('z15N50.csv','r')
L1=[float(ii) for ii in f1]
n=len(L1)


# Modelling 
mint =pm.Beta('Mint',alpha=1,beta=1)
coin0=pm.Bernoulli('coin01',p=mint,value=L1,observed=True)

M0=pm.Model([mint,coin0])


# Fitting 
M=pm.MCMC(M0) # == pm.MCMC([mint,coin0])
M.sample(iter=10000,burn=1000,thin=10)


# Print 
M.summary()
pm.Matplot.plot(M)

 [-----------------100%-----------------] 10000 of 10000 complete in 0.3 sec
Mint:
 
	Mean             SD               MC Error        95% HPD interval
	------------------------------------------------------------------
	0.309            0.063            0.002            [ 0.196  0.441]
	
	
	Posterior quantiles:
	
	2.5             25              50              75             97.5
	 |---------------|===============|===============|---------------|
	0.197            0.267           0.303          0.35          0.444
	
Plotting Mint


5 장에서는 입력값을 각 노드로 셋팅할 때의 방법에 대해서 간단하게 설명하고 있습니다. MCMC 명령에 직접 데이터를 연결하는 것도 방법이겠으나, pyMC 에서는 Model() 이라는 명령어로도 사용이 가능합니다.  


 - M = Model(set([a,b,c]))

 - M = Model({`a': a, `d': [b,c]}) 

 - M = Model([[a,b],c])

 - in case that MyModule.py has model 

In [ ]:
import MyModule
M = Model(MyModule)

  - Using a ‘model factory’ function

In [ ]:
def make_model(x):
    a = pymc.Exponential('a', beta=x, value=0.5)

    @pymc.deterministic
    def b(a=a):
        return 100-a

    @pymc.stochastic
    def c(value=.5, a=a, b=b):
        return (value-a)**2/b

    return locals()

M = pymc.Model(make_model(3))


  # 5.2 Model Class
  
   ####  Model() argument  : Model 명령어로 노드 들을 묶을 때
    - Input : 
     Some collection of PyMC nodes defining a probability model. These may be stored in a list, set, tuple,  
     dictionary, array, module, or any object with a __dict__ attribute.
    - Verbose : 
     An integer controlling the verbosity of the model’s output.
 
   #### ? Useful Method 
    - draw_from_prior():
     Sets all stochastic variables’ values to new random values, which would be a sample from the joint distribution 
     if all data and Potential instances’ log-probability functions returned zero. 
    - seed():
     Same as draw_from_prior, but only stochastics whose rseed attribute is not None are changed.
        
   #### Attributes
     variables
     nodes
     stochastics
     potentials
     deterministics
     observed_stochastics
     containers
     value
     logp
    
    
  # 5.3  Maximum a posteriori estimates¶
  
   Scipy 의 최적화 코드를 이용하여 maximum a posteriori values 를 계산할 수 있습니다. (SciPy는 설치 되어 있어야합니다.) float 에 대해서면 계산 가능하고  examples/gelman_bioassay.py using MAP (Nelder-Mead optimization) 을 계산 할 수 있습니다. 
        

In [9]:
from pymc.examples import gelman_bioassay
M = pymc.MAP(gelman_bioassay)
M.fit()  #fit(method ='fmin', iterlim=1000, tol=.0001): fmin, fmin_l_bfgs_b, fmin_ncg, fmin_cg, or fmin_powell
         #Scipy optimization page...

         #revert_to_max(): If the values of the constituent (...?) stochastic variables change after fitting, 
         # this function will reset them to their maximum a posteriori values. 

           
# Attributes        
#M.variables
M.nodes
#M.alpha.value 
#M.beta.value
#M.AIC
#M.BIC


{<pymc.CommonDeterministics.Lambda 'LD50' at 0x10aee4250>,
 <pymc.distributions.Binomial 'deaths' at 0x10af64d50>,
 <pymc.CommonDeterministics.Lambda 'theta' at 0x10af64dd0>,
 <pymc.distributions.Normal 'alpha' at 0x10aeeba10>,
 <pymc.distributions.Normal 'beta' at 0x10aeeb9d0>}

  # 5.4 Normal approximations
  
    The NormApprox class extends the MAP class by approximating the posterior covariance of the model using the Fisher information matrix, or the Hessian of the joint log probability at the maximum.
    

In [6]:
import pymc
from pymc.examples import gelman_bioassay

N = pymc.NormApprox(gelman_bioassay)
N.fit()


N.mu[N.alpha]
#N.mu[N.alpha, N.beta]
#N.C[N.alpha]
#N.C[N.alpha, N.beta]

#N.sample(100)
#N.trace('alpha')[::10]
#N.trace('beta')[::10]


array([ 0.65231548])

  # 5.5. Markov chain Monte Carlo: the MCMC class
  
  ###  sample(iter, burn, thin, tune_interval, tune_throughout, save_interval, ...):  계산 중간 중간에 연산된 값들을 저장. 
   
    iter : ontrols the total number of MCMC iterations. 
   
    the first burn iterations; these samples will be forgotten. After this burn-in period, 
    
    tallying will be done each thin iterations. 
    
    ? tuning will be done each tune_interval iterations. 
    
    ? tune_throughout=False, no more tuning will be done after the burnin period. 
    
    The model state will be saved every save_interval iterations, if given.
    

  #### isample(iter, burn, thin, tune_interval, tune_throughout, save_interval, ...):  The sampling loop may be paused  at any time, returning control to the user.
    
 
  #### use_step_method(method, *args, **kwargs):  Creates an instance of step method class method to handle some stochastic variables. 
    
    
  #### goodness():  Calculates goodness-of-fit (GOF) statistics according to [Brooks2000].
    

  #### save_state(): Saves the current state of the sampler, including all stochastics, to the database. This allows the sampler to be reconstituted at a later time to resume sampling. This is not supported yet for the sqlite backend.
  

  #### restore_state(): Restores the sampler to the state stored in the database. 

  #### stats(): Generate summary statistics for all nodes in the model.
  
  #### remember(trace_index): Set all variables’ values from frame trace_index in the database.  
  
  
 
 
 
 





# 5.6 Sampler Class
   ### MCMC 가 Sampler Class의 서브 클래스이고, sample(iter, length, verbose, ...) 명령으로 joint distribution에 대해 계산. 
   
   - sample(iter, length, verbose, ...):
     Samples from the joint distribution. The iter argument controls how many times the sampling loop will be run, and 
     the length argument controls the initial size of the database that will be used to store the samples.
   
   ### 그 외, isample(iter, length, verbose, ...), icontinue(), halt(), tally(), save_state(), restore_state(), stats(),  remember(trace_index) 의 옵션이 있습니다. 










# 5.7 Step Method
  
     Metropolis, AdaptiveMetropolis and Slicer 가 사용 가능하고, tune() 으로 계산 도중의 값들 및 셋팅 조절이 가능 
  
  
   ## 5.7.1  Metropolis step methods
   

In [ ]:
M.use_step_method(pymc.Metropolis, x, proposal_sd=1., proposal_distribution='Normal')

   - propose(): Sets the values of the variables handled by the Metropolis step method to proposed values.

   - reject(): If the Metropolis-Hastings acceptance test fails, this method is called to reset the values of the           variables to their values before propose() was called.

   - Note that there is no accept() method
   
   - proposal_sd: A float or array of floats. This sets the proposal standard deviation if the proposal distribution 
     is normal.
     

 
   ## 5.7.2  The AdaptiveMetropolis class
   - Its variables are block-updated using a multivariate jump distribution whose covariance is tuned during sampling
   - 따라서, Non Markovian 
     
   

In [10]:
M.use_step_method(pymc.AdaptiveMetropolis, [x,y,z], \
                   scales={'x':1, 'y':2, 'z':.5}, delay=10000)

AttributeError: 'MAP' object has no attribute 'use_step_method'

   ### 5.7.3~5.7.5 
   #### The DiscreteMetropolis class/  The BinaryMetropolis class / The Slicer class 도 계산 가능합니당. 

    







## 5.8. Gibbs step methods
     
    `parent’ 와  `children’의 요소로 나뉘어진 입력을 가집니다. 예를 들면, 정상 분포를 따르는 몇개의 변수의 조건이 a gamma-distributed variable 을 따른다고 할 때, gamma variable -> the parent, the normal variables -> the children. 
    

   ### 5.8.1. Granularity of step methods: one-at-a-time vs. block updating
    There is currently no way for a stochastic variable to compute individual terms of its log-probability; it is computed all together. This means that updating the elements of a array-valued variable individually would be inefficient, so all existing step methods update array-valued variables together, in a block update.

    To update an array-valued variable’s elements individually, simply break it up into an array of scalar-valued variables. Instead of this: An individual step method will be assigned to each element of A in the latter case, and the elements will be updated individually. Note that A can be broken up into larger blocks if desired.



   ### 5.8.2. Automatic assignment of step methods

    Every step method subclass (including user-defined ones) that does not require any __init__ arguments other than the stochastic variable to be handled adds itself to a list called StepMethodRegistry in the PyMC namespace. If a stochastic variable in an MCMC object has not been explicitly assigned a step method, each class in StepMethodRegistry is allowed to examine the variable.



   
   